In [5]:
import os
import re
import json
import pandas as pd
import math

conversion_path = 'wiki_conversion.csv'
patch_path = 'r2data_patch_113.json' # Make sure to change this line with a new patch update!
reader = 'readerscript.txt'
writer = 'writerscript.txt'

In [6]:
# Loads conversion data.

d = pd.read_csv(conversion_path)
# spamreader = csv.reader(translated_names)
import numpy as np
s = d.to_numpy(na_value='None')
new_d = dict()
for x in s:
    if x[3] == 'None':
        x[3] = x[0]
    if x[4] == 'None':
        x[4] = x[1]
    if x[5] == 'None':
        x[5] = x[2]
    if(x[3] not in new_d.keys()):
        new_d[x[3]] = dict()
    if(x[4] not in new_d[x[3]].keys()):
        new_d[x[3]][x[4]] = dict()
    if(x[5] not in new_d[x[3]][x[4]].keys()):
        new_d[x[3]][x[4]][x[5]] = dict()
    new_d[x[3]][x[4]][x[5]] = [*x[0:3], *[x[6]]]
move_name_conversion = {'Bair': 'Back Air',
 'Bthrow': 'Back Throw',
 'Dair': 'Down Air',
 'DashGrab': 'Dash Grab',
 'Dattack': 'Dash Attack',
 'Dspecial': 'Down Special',
 'Dstrong': 'Down Strong',
 'Dthrow': 'Down Throw',
 'Dtilt': 'Down Tilt',
 'Fair': 'Forward Air',
 'Fspecial': 'Forward Special',
 'Fstrong': 'Forward Strong',
 'Fthrow': 'Forward Throw',
 'Ftilt': 'Forward Tilt',
 'GetupAttack': 'Getup Attack',
 'GetupSpecial': 'Getup Special',
 'Grab': 'Grab',
 'Jab': 'Jab',
 'LedgeAttack': 'Ledge Attack',
 'LedgeSpecial': 'Ledge Special',
 'Nair': 'Neutral Air',
 'Nspecial': 'Neutral Special',
 'PivotGrab': 'Pivot Grab',
 'Pummel': 'Pummel',
 'PummelSpecial': 'Special Pummel',
 'Uair': 'Up Air',
 'Uspecial': 'Up Special',
 'Ustrong': 'Up Strong',
 'Uthrow': 'Up Throw',
 'Utilt': 'Up Tilt'}
reversed = dict()
for i in list(move_name_conversion.items()):
    reversed[i[1]] = i[0]

In [7]:
# This loads all the functions necessary for processing game data and converting it to wiki template data.

def print_thing(atk):
    base_string = ''
    new_move_name = atk[1]
    if(new_move_name in reversed.keys()):
        new_move_name = reversed[atk[1]]
    if(atk[2] == 'Throw'):
        v = char_data[atk[0]][new_move_name]['Throws'][0]
        return print_throw(atk[0], new_move_name, v)
    else:
        v = char_data[atk[0]][new_move_name]['KBs'][atk[2]]
        return print_hit(atk[0], new_move_name, atk[2], v, atk[3])
def print_throw(character, move, v):
    strung = '| damage         = ' + str(v['damage']) + '%\n'
    strung += '| angle          = ' + str(v['angle']) + '\n'
    strung += '| baseKb         = ' + str(v['base_kb']) + '\n'
    strung += '| kbScale        = ' + str(v['kb_scale']) + '\n'
    strung += '| isThrowFlag                 = True\n'
    strung += '| groundTechableFlag          = ' + str(v['ground_techable']) + '\n'
    strung += '| weightIndependentFlag       = True' + '\n'
    strung += '| hitstunMulti                = ' + str(v['hitstun']) + '\n'
    strung += '| totalShieldStun             = N/A\n'        
    return strung
def print_hit(character, move, hit, v, hit_count):
    # print(hit_count)
    strung = ''
    
    strung += '| damage             = ' + str(v['damage']) + '%'
    if(hit_count != 'None'):
        strung += ' (' + str(hit_count) + 'x)'
    strung += '\n'
    strung += '| baseKb             = ' 
    if(v['final_base_kb'] > 0):
        strung += '{{tt|*|' + str(float(v['base_kb'])) + ' to ' + str(v['final_base_kb']) + ' over time.}} \n'
    else:
        strung += str(v['base_kb']) + '\n'
    strung += '| kbScale            = ' + str(v['kb_scale']) + '\n'
    strung += '| angle              = ' + str(v['angle']) + '\n'
    strung += '| autoFloorhugFlag   = ' + str(v['auto_floorhug']) + '\n'

    strung += '| isProjectileFlag   = ' + str(char_data[character][move]['ExtraProps']['isProjectileFlag']) + '\n'
    strung += '| isArticleFlag      = ' + str(char_data[character][move]['ExtraProps']['isArticleFlag']) + '\n'
    if 'bReverseCat' in char_data[character][move]['ExtraProps'].keys():
        strung += '| bReverseCat        = ' + str(char_data[character][move]['ExtraProps']['bReverseCat'].split('::')[-1]) + '\n'

    strung += '| hitpauseMulti                  = ' + str(v['hitpause_multi']) + '\n'
    strung += '| extraOppHitpause               = ' + str(v['extra_hitpause']) + '\n'
    hitpause_amount = math.floor(v['damage'] * v['hitpause_multi'])
    total_hp = max(min(hitpause_amount, 24),3)
    strung += '| totalHitpause                  = ' + str(total_hp)
    if(v['extra_hitpause'] > 0):
        strung += ' {{tt|[' + str(hitpause_amount + v['extra_hitpause']) + ']|Opponent hitpause.}}'
    strung += '\n'
    strung += '| hitpauseMovementStrength       = ' + str(v['hitpause_multi']) + '\n'
    strung += '| ssdiMulti                      = ' + str(v['sdi_multi']) + '\n'
    if(v['asdi_multi'] == -1.0):
        strung += '| asdiMulti                      = ' + str(v['sdi_multi']) + '\n'
    else:
        strung += '| asdiMulti                      = ' + str(v['asdi_multi']) + '\n'
    strung += '| reverseHitFlag                 = ' + str(v['reverse_flag']) + '\n'
    strung += '| forceFlinchFlag                = ' + str(v['force_flinch']) + '\n'
    strung += '| groundTechableFlag             = ' + str(v['ground_techable']) + '\n'
    strung += '| breakProjectileFlag            = ' + str(v['breaks_projectiles']) + '\n'
    strung += '| weightIndependentFlag          = ' + str(v['weight_independent']) + '\n'
    strung += '| knockbackFlipper               = ' + str(v['kb_flipper']) + '\n'
    strung += '| hitstunMulti                   = ' + str(v['hitstun']) + '\n'
    strung += '| hitfallHitstunMulti            = ' + str(v['hitfall_stun']) + '\n'
    
    if(v['parry_stun'] == 'NoStun'):
        strung += '| parryReaction                  = No Stun'
    else:
        strung += '| parryReaction                  = ' + v['parry_stun']
    if(char_data[character][move]['ExtraProps']['isArticleFlag']):
       strung += ' (' + char_data[character][move]['ExtraProps']['ParryReaction'] + ')'
    strung += '\n'
    strung += '| grabPartnerInteraction         = ' + str(v['grab_partner']) + '\n'

    strung += '| extraShieldStun                = ' + str(v['extra_shield_stun']) + '\n'
    strung += '| totalShieldStun                = ' + str(v['shield_stun']) + '\n'

    strung += '| shieldDamageMulti              = ' + str(v['shield_damage']) + '\n'
    strung += '| totalShieldDamage              = ' + str("{:.2f}\n".format((v['shield_damage'] * (0.0443*v['damage']*v['damage']+0.6831*v['damage']+0.1244))))

    strung += '| shieldPushbackMulti            = ' + str(v['shield_push']) + '\n'
    strung += '| totalShieldPushback            = ' + str("{:.3f}\n".format(v['shield_push']*0.8*v['damage'] + 2))

    strung += '| shieldHitpauseMulti            = ' + str(v['shield_hitpause']) + '\n'
    if(char_data[character][move]['ExtraProps']['isProjectileFlag']):
        strung += '| totalShieldHitpause            = ' + str(math.floor(total_hp / 2 * v['shield_hitpause'])) + '\n'
    else:
        strung += '| totalShieldHitpause            = ' + str(math.floor(total_hp * v['shield_hitpause'])) + '\n'

    strung += '| fullChargeKbMulti              = ' + str(v['fully_charged_kb_multi']) + '\n'
    strung += '| fullChargeDamageMulti          = ' + str(v['fully_charged_dmg_multi']) + '\n'
    strung += '| finalBaseKb                    = ' + str(v['final_base_kb']) + '\n'
    strung += '| forceTumbleFlag                = ' + str(v['force_tumble']) + '\n'
    # strung += '| ignoreKnockbackArmor                = ' + str(v['fully_charged_kb_multi']) + '\n'
    return strung

Pay attention to this bottom cell here.

In [8]:
# This takes the wiki page sitting in readerscript.txt, sifts through all of the game data, and overwrites any existing values. It'll usually be correct.
# If an error occurs, that means that the wiki template name cannot be found in the wiki_conversion.csv file. Oops! Just adjust it in the wiki_conversion.csv file, save it, and then run all the previous cells and this cell again. 
# Make sure that both the readerscript and writerscript files are both saved before proceeding. It can get ugly with unsaved files.

# Once you've loaded all the previous cells, if there are no errors, you only need to rerun this cell every time for each character!

char_data = json.load(open(patch_path, 'r'))
w = open(writer, 'w')
with open(reader) as f:
    e = f.readline()
    start_writing = True
    atk = {'char':'Loxodont', 'atk':'', 'hit':''}
    while(e):
        if(re.match('\\| chara =.+',e)):
            atk['char'] = e.split('=')[1].strip()
        if(re.match('\\| attack\\s+=.+',e)):
            atk['atk'] = e.split('=')[1].strip()
        elif(re.match('\\| name\\s+=.+',e)):
            atk['hit'] = e.split('=')[1].strip()
        elif(re.match('\\| damage\\s+=.+',e)):
            start_writing = False
            print(atk)
            w.write(print_thing(new_d[atk['char']][atk['atk']][atk['hit']]))
        if(re.match('}}\n', e)):
            start_writing = True
        if(re.match('\\| notes\\s+=.+', e)):
            w.write(e)
        elif(start_writing):
            w.write(e)
        
        e = f.readline()
        # print(atk)

    f.close()
w.close()

{'char': 'Orcane', 'atk': 'Jab', 'hit': 'Jab 1'}
{'char': 'Orcane', 'atk': 'Jab', 'hit': 'Jab 2'}
{'char': 'Orcane', 'atk': 'Jab', 'hit': 'Jab 3'}
{'char': 'Orcane', 'atk': 'Up Tilt', 'hit': 'Sweet'}
{'char': 'Orcane', 'atk': 'Up Tilt', 'hit': 'Sour'}
{'char': 'Orcane', 'atk': 'Forward Tilt', 'hit': 'Ftilt'}
{'char': 'Orcane', 'atk': 'Down Tilt', 'hit': 'Sweet'}
{'char': 'Orcane', 'atk': 'Down Tilt', 'hit': 'Sour'}
{'char': 'Orcane', 'atk': 'Dash Attack', 'hit': 'Early'}
{'char': 'Orcane', 'atk': 'Dash Attack', 'hit': 'Late'}
{'char': 'Orcane', 'atk': 'Up Strong', 'hit': 'Normal Sweet'}
{'char': 'Orcane', 'atk': 'Up Strong', 'hit': 'Normal Sour'}
{'char': 'Orcane', 'atk': 'Up Strong (Puddle)', 'hit': 'Puddle Sweet'}
{'char': 'Orcane', 'atk': 'Up Strong (Puddle)', 'hit': 'Puddle Sour'}
{'char': 'Orcane', 'atk': 'Forward Strong', 'hit': 'Normal'}
{'char': 'Orcane', 'atk': 'Forward Strong (Puddle)', 'hit': 'Puddle'}
{'char': 'Orcane', 'atk': 'Down Strong', 'hit': 'Normal'}
{'char': 'Orcan